In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as mplot
from matplotlib.colors import ListedColormap
import os
import subprocess
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score,  precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

import shap
import lime
from lime import lime_tabular
import random

from sklearn.preprocessing import StandardScaler

from sklearn.tree import export_graphviz
import pydotplus
import seaborn as sns


import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.utils.vis_utils import model_to_dot



In [ ]:
import tensorflow as tf

print(tf.__version__)
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU'))

import dalex as dx
dx.__version__

import torch


## Reading Dataset, Preprocessing and making X and Y variables

In [ ]:
dataSetIndex = 6
needToMakePictureOfTrees = 0
''' ---------------------------------------------------------- '''
dataSetFilePath = ""
dataSetName = ""
dataSetResultDirectory = "./"

if(dataSetIndex == 0):
    dataSetFilePath = "./heartDisease/0_statLog_dataSet.csv"
    dataSetName = "SateLog_DataSet"
elif (dataSetIndex == 1):
    dataSetFilePath = "./heartDisease/1_heart_statlog_cleveland_hungary_final.csv"
    dataSetName = "ALL_StateLog_CleveLand_Hungary"
elif (dataSetIndex == 2):
    dataSetFilePath = "./heartDisease/2_cleveland.csv"
    dataSetName = "Cleveland"
elif (dataSetIndex == 3):
    dataSetFilePath = "./heartDisease/3_framingham.csv"
    dataSetName = "framingham"
elif (dataSetIndex == 4):
    dataSetFilePath = "./heartDisease/4_CardiacPrediction.xlsx"
    dataSetName = "CardiacPrediction"
elif (dataSetIndex == 5):
    dataSetFilePath = "./heartDisease/5_CardiacPredictionLessDimensions.xlsx"
    dataSetName = "CardiacPredictionLessDimensions"
elif (dataSetIndex == 6):
    dataSetFilePath = "./heartDisease/6_CardiacPredictionFewDimensions.xlsx"
    dataSetName = "CardiacPredictionFewDimensions"
else:
    dataSetFilePath = ""
    dataSetName = ""

if(dataSetIndex==4 or dataSetIndex==5 or dataSetIndex==6):
    #fileData = pd.read_excel(dataSetFilePath, sheet_name='CoroHeartDis')
    fileData = pd.read_excel(dataSetFilePath)
else:
    fileData = pd.read_csv(dataSetFilePath)

print("Shape of fileData: {}".format(fileData.shape))
print("Column Headings: {}".format(fileData.__dataframe__().column_names()))
print("Number of Records: {}".format(fileData.__dataframe__().num_rows()))



num_rows_before = fileData.shape[0]
# Remove duplicate records based on all columns
fileData.drop_duplicates(inplace=True)
# Check the number of rows after removing duplicates
num_rows_after = fileData.shape[0]
# Print the number of duplicate records removed
num_duplicates_removed = num_rows_before - num_rows_after
print(f"Number of duplicate records removed: {num_duplicates_removed}")
 
 # Preprocess Steps from the ChatGPT
# 1. Handling Missing Values:
fileData = fileData.dropna()
print("Shape of fileData: {}".format(fileData.shape))              
#fileData.replace({'?': np.nan}).dropna().astype(float)
#fileData = fileData.fillna(0)

'''
for columnName in fileData.__dataframe__().column_names():
    #print("Column Name: {}, Len:{} ".format(columnName,len(fileData[columnName]))) 
    fileData[columnName] = np.floor(pd.to_numeric(fileData[columnName], errors='coerce')).astype('Int64') 
    meanValue = np.mean(fileData[columnName])
    print("Column Name: {}, means:{} ".format(columnName, meanValue))
    for i in range( len(fileData[columnName]) ):
        print("----> Column Name: {}, Index:{} , Value:{} ".format(columnName, i, (fileData[columnName][i]) ))
            #print("%%%%----> Column Name: {}, Index:{} , Value:{} \n\n".format(columnName, i, str(fileData[columnName][i]) ))
            fileData[columnName][i] = meanValue
'''

fileData = fileData.fillna(0)
# 2. Handling Outliers:
# 3. Feature Scaling:
# 4. Encoding Categorical Variables:
# 5. Feature Engineering:
# 6. Handling Skewed Distributions:
# 7. Dimensionality Reduction:
# 8. Handling Imbalanced Classes:
# 9. Normalization of Data:
# 10. Data Splitting:

print("Shape of fileData End: {}".format(fileData.shape))



dataSetResultDirectory += dataSetName
dataSetResultDirectory += "/"
if not os.path.isdir(dataSetResultDirectory):
    os.makedirs(dataSetResultDirectory)

dataSetName += " {}".format(fileData.shape)

if(dataSetIndex == 0): 
    X = fileData[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]  # Features
    Y = fileData['target']  # Labels
elif (dataSetIndex == 1): 
    #X = fileData[['age', 'sex', 'chest pain type', 'resting bp s', 'cholesterol', 'fasting blood sugar', 'resting ecg', 'max heart rate', 'exercise angina', 'oldpeak', 'ST slope']]  # Features
    X = fileData[['fasting blood sugar', 'resting ecg', 'max heart rate', 'exercise angina', 'oldpeak']]  # Features LIME KNN
    Y = fileData['target']  # Labels
elif(dataSetIndex == 2): 
    X = fileData[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]  # Features
    Y = fileData['target']  # Labels
elif(dataSetIndex == 3): 
    X = fileData.drop('TenYearCHD', axis=1)  # Features
    Y = fileData['TenYearCHD']  # Labels
elif(dataSetIndex == 4): 
    X = fileData.drop('CoronaryHeartDisease', axis=1)  # Features
    Y = fileData['CoronaryHeartDisease']  # Labels
else: 
    X = fileData.drop('CoronaryHeartDisease', axis=1)  # Features
    Y = fileData['CoronaryHeartDisease']  # Labels



print("\n")
print("columns of x:: {} \n\n and features of X: {}".format(len(X.columns), X.columns))




## Making Training 70% and Testing 30%

In [ ]:
print("\n")
print("columns of x:: {} \n\n and features of X: {}".format(len(X.columns), X.columns))
 
print("***************************************") 

print("Shape of fileData: {} , target Len:{}".format(fileData.shape, len(Y)))
print("X: {} , Y:{}".format(X.shape, Y.shape))
#print("\n\nX: head:: \n{}".format(X.head()))
#print("\n\nY: head::\n {}".format(Y.head()))

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3) # 70% training and 30% test

print("\n X Train: Shape::\n {}".format(X_train.shape))
print("\n X Train: head::\n {}".format(X_train.columns))
print("\n X Test: head:: \n{}".format(X_test.columns))
print("\n Y Train: shape::\n {}".format(y_train.shape)) 
print("\n Y Test: shape::\n {}".format(y_test.shape)) 
 
instance = np.array(X_test)  # Example: explaining the first instance in the dataset



In [ ]:
n, p = X.shape

tf.random.set_seed(11)

normalizer  = tf.keras.layers.experimental.preprocessing.Normalization(input_shape=[p,])
normalizer.adapt(X.to_numpy())

model = tf.keras.Sequential([
    normalizer,
    tf.keras.Input(shape=(p,)),
    tf.keras.layers.Dense(p*2, activation='relu'),
    tf.keras.layers.Dense(p*3, activation='relu'),
    tf.keras.layers.Dense(p*2, activation='relu'),
    tf.keras.layers.Dense(p, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.mae
)



In [ ]:
model.fit(X, Y, batch_size=int(n/10), epochs=2000, verbose=False)


In [ ]:
explainer = dx.Explainer(model, X, Y, label='CoronaryHeartDisease')


In [ ]:
explainer.model_performance()
explainer.model_parts().plot()


In [ ]:

'''
Shape of fileData: (37079, 51)
Column Headings: Index(['SEQN', 'Gender', 'Age', 'Annual-Family-Income',
       'Ratio-Family-Income-Poverty', 'X60-sec-pulse', 'Systolic', 'Diastolic',
       'Weight', 'Height', 'Body-Mass-Index', 'White-Blood-Cells',
       'Lymphocyte', 'Monocyte', 'Eosinophils', 'Basophils', 'Red-Blood-Cells',
       'Hemoglobin', 'Mean-Cell-Vol', 'Mean-Cell-Hgb-Conc.',
       'Mean-cell-Hemoglobin', 'Platelet-count', 'Mean-Platelet-Vol',
       'Segmented-Neutrophils', 'Hematocrit', 'Red-Cell-Distribution-Width',
       'Albumin', 'ALP', 'AST', 'ALT', 'Cholesterol', 'Creatinine', 'Glucose',
       'GGT', 'Iron', 'LDH', 'Phosphorus', 'Bilirubin', 'Protein', 'Uric.Acid',
       'Triglycerides', 'Total-Cholesterol', 'HDL', 'Glycohemoglobin',
       'Vigorous-work', 'Moderate-work', 'Health-Insurance', 'Diabetes',
       'Blood-Rel-Diabetes', 'Blood-Rel-Stroke', 'CoronaryHeartDisease'],
'''

#explainer.predict_parts(X_train[50], type='shap').plot()
lime_explanation = explainer.predict_surrogate(X_train.loc[1], mode='regression')



## Methods Definitions

In [ ]:
def plot_classification_report(title, dataSetName, y_tru, y_prd, figsize=(6, 6), ax=None):
    #mplot.figure(figsize=figsize)
    xticks = ['precision', 'recall', 'f1-score', 'support']
    yticks = ['Healthy', 'Heart Disease']
    rep = np.array( precision_recall_fscore_support(y_tru, y_prd) ).T
    rep[0][0] *= 100.0
    rep[0][1] *= 100.0
    rep[0][2] *= 100.0
    rep[1][0] *= 100.0
    rep[1][1] *= 100.0
    rep[1][2] *= 100.0
    
    ax = sns.heatmap(rep, annot=True, cmap='Blues', cbar=False, xticklabels=xticks, yticklabels=yticks)
    ax.set_title("Classification Report {} Model\n\n".format(title) ,fontsize=16, fontweight='bold')
    ax.set_xlabel('\nDataset:{}'.format(dataSetName))
    ax.xaxis.set_ticklabels(xticks)
    ax.set_ylabel('Classes')
    ax.yaxis.set_ticklabels(yticks)
    
    picturePath = "{}ClassificationReport_{}_{}.png".format(dataSetResultDirectory, title, dataSetName) 
    mplot.savefig(picturePath, dpi=300, bbox_inches='tight')
    #os.startfile(picturePath)
    mplot.close()

def makeConfusionMatrixPic(method, dataSet, classifierObj , X_test, y_test, predicted_Y):
    display = ConfusionMatrixDisplay.from_estimator(classifierObj, X_test, y_test, display_labels=['Healthy', "Heart Disease"], cmap=mplot.cm.Blues) #, normalize="true"
    display.ax_.set_title("Confusion Matrix ({} Model)".format(method) ,fontsize=16, fontweight='bold')
    display.ax_.set_xlabel('\nPredicted Values')
    display.ax_.set_ylabel('Actual Values ')

    dataSetString = "Dataset: {}".format(dataSet)
    accuracyString ="Accuracy {}: {}".format(method, accuracy_score(y_test, predicted_Y)*100.0 ) 
    recallString =  'Recall {}: {}'.format(method, recall_score(y_test, predicted_Y) * 100.0)
    #precisionString = 'Precision {}: {}'.format(method, precision_score(y_test, predicted_Y, average='weighted') * 100.0) 
    precisionString = 'Precision {}: {}'.format(method, precision_score(y_test, predicted_Y) * 100.0) 
    display.figure_.text(0.020, -0.05,  accuracyString, horizontalalignment='left', wrap=False )  
    display.figure_.text(0.020, -0.09,  recallString, horizontalalignment='left', wrap=False )      
    display.figure_.text(0.020, -0.13,  precisionString, horizontalalignment='left', wrap=False ) 

    display.figure_.text(0.020, -0.18,  dataSetString, horizontalalignment='left', wrap=False ) 
 
    picturePath = "{}Confusion_Matrix_{}_{}.png".format(dataSetResultDirectory, method, dataSetName)
    mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
    #print("{} Confusion Matrix saved:: path: {}".format(method, picturePath))
    #os.startfile(picturePath)
    mplot.show()
    mplot.close()

In [ ]:
# Deep Learning Model
kerasSequentialModel = Sequential();

kerasSequentialModel.add(Dense(13,  input_dim=13, activation='relu'))
kerasSequentialModel.add(Dense(8, activation='relu'))
kerasSequentialModel.add(Dense(1, activation='sigmoid'))

# loss='mean_sequared_error'
#kerasSequentialModel.compile(optimizer="adam", loss='binary_crossentropy')
kerasSequentialModel.compile(optimizer="adam", loss='mean_squared_error')
 


In [ ]:
#kerasSequentialModel.fit(X_train, y_train, epochs=10, batch_size=10 )
# Train model
#kerasSequentialModel.fit(X_train, y_train, batch_size=30, epochs=10, verbose=False, validation_data=(X_test, y_test))
kerasSequentialModel.fit(np.asarray(X_train).astype('float32') , np.asarray(y_train).astype('float32'), epochs=310, batch_size=10)


In [ ]:
#np.asarray(X_train).astype('float32')
#score = kerasSequentialModel.evaluate(np.asarray(X_test).astype('float32'), np.asarray(y_test).astype('float32'), verbose=0)
score = kerasSequentialModel.evaluate(np.asarray(X_test).astype('float32'), np.asarray(y_test).astype('float32') )
print('Test Score:', score)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Summary of neural network
kerasSequentialModel.summary()

In [ ]:

predictionsKerasSequential = kerasSequentialModel.predict(np.asarray(X_test).astype('float32'))

print('Test Score:', predictionsKerasSequential)

methodName = "Keras Sequential Model" 
#plot_classification_report(methodName, dataSetName, y_test, predictionsKerasSequential)  
makeConfusionMatrixPic(methodName, dataSetName, kerasSequentialModel , np.asarray(X_test).astype('float32'), np.asarray(y_test).astype('float32'), predictionsKerasSequential) 
  
confusionMatric = confusion_matrix(y_test, predictionsKerasSequential)
print("{} ConfusionMatrix: {}".format(methodName, confusionMatric))
print("Accuracy {}:  {}".format(methodName, accuracy_score(y_test, predictionsKerasSequential)))

In [ ]:

 
#fpr, tpr, thresholds = roc_curve(y_test, classifierLR.predict_proba(X_test)[:,1])
mplot.figure()
#mplot.plot(fpr, tpr, label="(area = {:.2f})".format(logit_roc_auc))

fprLR, tprLR, threshLR = roc_curve(y_test, classifierLR.predict_proba(X_test)[:,1])
#fprSVM, tprSVM, threshSVM = roc_curve(y_test, classifierSVM.predict_proba(X_test)[:,1])
fprKNN, tprKNN, threshKNN = roc_curve(y_test, classifierKNN.predict_proba(X_test)[:,1])
fprNB, tprNB, threshNB = roc_curve(y_test, classifierNB.predict_proba(X_test)[:,1])
 
    
# plotting    
mplot.plot(fprLR, tprLR, linestyle='--',color='green', label='Class 1 vs Rest')
#mplot.plot(fprSVM, tprSVM, linestyle='--',color='orange', label='Class 0 vs Rest')
mplot.plot(fprKNN, tprKNN, linestyle='--',color='blue', label='Class 2 vs Rest')
mplot.plot(fprNB, tprNB, linestyle='--',color='red', label='Class 2 vs Rest')
mplot.title('Multiclass ROC curve')
mplot.xlabel('False Positive Rate')
mplot.ylabel('True Positive rate')
mplot.legend(loc='best')
mplot.savefig('Multiclass ROC',dpi=300);    
mplot.show